# O Problema
Sliding Puzzle - Bloco Deslizante

In [3]:
# !wget -qq https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif
from IPython.display import Image
Image(url='https://miro.medium.com/max/700/1*W7jg4GmEjGBypd9WPktasQ.gif',width=200)

In [4]:
import numpy as np

class Puzzle:
  def __init__(self, board=None, min_moves=5, max_moves=20):  
    if board is None:
      self.board = np.array([[1,2,3],[4,5,6],[7,8,-1]])
      self.empty = (2,2)

      self.shuffle(min_moves, max_moves)
    else:
      self.board = board
      e = np.where(board == -1)
      self.empty = (e[0][0], e[1][0])
        
  def print_board(self):
    print(self.board)
        
  def get_allowed_moves(self):
    allowed_moves = []
    
    allowed_moves += self.get_vertical_moves()
    allowed_moves += self.get_horizontal_moves()

    np.random.shuffle(allowed_moves)     # usando um 'shuffle' pra evitar loops na busca em profundidade
    return allowed_moves

    
  def get_vertical_moves(self):
    allowed_moves = []
    if self.empty[0] > 0:
      allowed_moves.append((self.empty[0] - 1, self.empty[1])) # down
    if self.empty[0] < len(self.board) - 1:
      allowed_moves.append((self.empty[0] + 1, self.empty[1])) # up
        
    return allowed_moves
          
  def get_horizontal_moves(self):
    allowed_moves = []
    if self.empty[1] > 0:
      allowed_moves.append((self.empty[0], self.empty[1] - 1)) # right
    if self.empty[1] < len(self.board) - 1:
      allowed_moves.append((self.empty[0], self.empty[1] + 1)) # left
          
    return allowed_moves
  

    
  def move(self, piece_to_move):
    self.board[self.empty] = self.board[piece_to_move]
    self.board[piece_to_move] = -1
      
    self.empty = piece_to_move
        
    
    
  def check_puzzle(self):
    last = 0
    for i, lin in enumerate(self.board):
      for j, col in enumerate(lin):
        # Se chegar na última posição, tá ordenado
        if i == len(self.board) - 1 and  j == len(self.board) - 1:
          return True
              
        # Se alguma peça é menor que a anterior, não tá ordenado
        if col < last:
          return False
        last = col
                
                
  def shuffle(self, min_moves, max_moves):
    rng = np.random.default_rng()
    num_of_moves = rng.integers(min_moves, max_moves)

    for i in range(num_of_moves):
      allowed_moves = self.get_allowed_moves()
      move = rng.integers(0, len(allowed_moves))

      self.move((allowed_moves[move]))


In [5]:
class Tree:
  def __init__(self, movement, puzzle, parent=None):
    self.movement = movement
    self.puzzle = puzzle
    self.children = puzzle.get_allowed_moves()
    self.parent = parent

    self.visited = False

  def create_child(self, i): # cria o filho i (adiciona o puzzle, executa o movimento, cria a árvore)
    p = Puzzle(self.puzzle.board.copy())
    p.move(self.children[i])

    self.children[i] = Tree(self.children[i], p, self)

# Resolver o quebra-cabeças usando Buscas

In [6]:
from collections import deque

In [7]:
def create_path(node):
  q = deque()
  q.append(node)

  while node.parent is not None:
    node = node.parent
    q.appendleft(node)

  return q

## Busca em largura

In [8]:
def bf_search(tree):
  q = deque()
  q.append(tree)

  while q: # while q is not empty
    node = q.popleft()

    for i, c in enumerate(node.children):
      node.create_child(i)
      q.append(node.children[i])

      if node.children[i].puzzle.check_puzzle():
        return node.children[i], create_path(node.children[i])

In [9]:
p = Puzzle(min_moves=10, max_moves=15)
t = Tree((-1,-1), p)

p.print_board()

n, path = bf_search(t)

[[ 1 -1  3]
 [ 4  2  6]
 [ 7  5  8]]


In [10]:
for p in path:
  p.puzzle.print_board()

[[ 1 -1  3]
 [ 4  2  6]
 [ 7  5  8]]
[[ 1  2  3]
 [ 4 -1  6]
 [ 7  5  8]]
[[ 1  2  3]
 [ 4  5  6]
 [ 7 -1  8]]
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8 -1]]


## Busca em Profundidade

In [11]:
def df_search(node):
  q = deque()
  node.visited = True
  q.append(node)

  while q:
    node = q[-1]

    for i in range(len(node.children)):
      if isinstance(node.children[i], tuple): 
        node.create_child(i)
      n = node.children[i]

      if not n.visited:
        q.append(n)
        n.visited = True

        if n.puzzle.check_puzzle():
          return n, create_path(n)

        break

      if i == len(node.children - 1):
        q.pop()

In [12]:
p = Puzzle(min_moves=3, max_moves=5)
t = Tree((-1,-1), p)

p.print_board()

n, path = df_search(t)

[[ 1  2  3]
 [ 4  5  6]
 [ 7 -1  8]]


In [13]:
len(path)

4

In [14]:
for p in path:
  p.puzzle.print_board()

[[ 1  2  3]
 [ 4  5  6]
 [ 7 -1  8]]
[[ 1  2  3]
 [ 4  5  6]
 [-1  7  8]]
[[ 1  2  3]
 [ 4  5  6]
 [ 7 -1  8]]
[[ 1  2  3]
 [ 4  5  6]
 [ 7  8 -1]]


## Discusta sobre o desempenho dos métodos em questões de:


1.   Consumo de memória
2.   Processamento



Busca em Largura: Gasta mais memória, porque precisa gerar todos movimentos, de todos ramos, feitos até a profundidade em que encontrar a melhor solução.
    A vantagem é que, como explora todos os movimentos, sempre vai encontrar o melhor caminho.

Busca em Profundidade: (Em teoria) gasta menos memória, já que armazena somente os movimentos executados na busca atual... Porém, como não possui uma forma de armazenar os estados já alcançados, ele não garante que não ocorrerão loops, podendo não obter resultado. 
    Caso encontre uma solução, não garante que é a melhor (com menor número de passos executados)

Para garantir que sempre será encontrada alguma solução, é necessário armazenar todos estados já encontrados.